In [ ]:
import os
import json
import glob
import time
import base64
import csv
import numpy as np
import cv2

LABEL_DIR =  #Label File
IMG_DIR   =       #Validation folder
LABEL_NAME = None 

OUT_CSV = os.path.join(LABEL_DIR, "eval_final_perframe.csv")
OUT_TXT = os.path.join(LABEL_DIR, "eval_final_summary.txt")

CANNY_LOW = 70
CANNY_HIGH = 190
BOUNDARY_TOL = 2 

EVAL_BAND = 8

BG_EXCLUDE_DILATE = 10

MEDIAN_KSIZE = 3  
M = 1326.395647  
K = 0.00128633    
C = -1368.862781  

GROW_THRESH = 1 

BOUNDARY_THICKNESS = 2

PROGRESS_STEP = 10  

def to_gray_u8(img):
    if img is None:
        return None
    # If already grayscale
    if len(img.shape) == 2:
        g = img
    else:
        # Convert BGR to grayscale
        g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Convert 16-bit to 8-bit if needed
    if g.dtype == np.uint8:
        return g
    if g.dtype == np.uint16:
        return (g / 256).astype(np.uint8)
    
    # Normalize to 0-255 if not 8-bit
    return cv2.normalize(g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def load_img(jd, json_path):
    img_data = jd.get("imageData", None)
    if img_data:
        try:
            data = base64.b64decode(img_data)
            arr = np.frombuffer(data, dtype=np.uint8)
            img = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)
            if img is not None:
                return img, None
        except Exception as e:
            pass

    imagePath = jd.get("imagePath", "")
    exts = (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff")
    candidates = []
    
    if imagePath:
        candidates.append(os.path.basename(imagePath))
    base = os.path.splitext(os.path.basename(json_path))[0]
    candidates.append(base)

    for c in candidates:
        if c.lower().endswith(exts):
            p = os.path.join(IMG_DIR, c)
            if os.path.exists(p):
                img = cv2.imread(p, cv2.IMREAD_UNCHANGED)
                if img is not None:
                    return img, p

    for c in candidates:
        b0 = os.path.splitext(c)[0]
        for ext in exts:
            p = os.path.join(IMG_DIR, b0 + ext)
            if os.path.exists(p):
                img = cv2.imread(p, cv2.IMREAD_UNCHANGED)
                if img is not None:
                    return img, p

    for c in candidates:
        b0 = os.path.splitext(c)[0]
        hits = []
        for ext in exts:
            hits += glob.glob(os.path.join(IMG_DIR, b0 + "*" + ext))
        if hits:
            img = cv2.imread(hits[0], cv2.IMREAD_UNCHANGED)
            if img is not None:
                return img, hits[0]

    return None, None

def resolve_label_name(jd, preferred):
    labels = []
    for sh in jd.get("shapes", []):
        lab = sh.get("label", None)
        if lab is not None and lab.strip():
            labels.append(lab.strip())
    
    uniq = sorted(set(labels))
    if not uniq:
        return None
    
    if preferred is None:
        return uniq[0]
    if preferred in uniq:
        return preferred
    return uniq[0]

def ensure_closed(pts_xy):
    if len(pts_xy) < 3:
        return None
    
    p0 = pts_xy[0]
    pL = pts_xy[-1]
    if (abs(p0[0] - pL[0]) + abs(p0[1] - pL[1])) > 1e-6:
        pts_xy = pts_xy + [p0]
    return pts_xy

def pts_to_int(pts_xy, h, w):
    poly = np.array(pts_xy, dtype=np.float32)
    poly = np.round(poly).astype(np.int32)
    # Clip to image boundaries
    poly[:, 0] = np.clip(poly[:, 0], 0, w - 1)
    poly[:, 1] = np.clip(poly[:, 1], 0, h - 1)
    return poly

def mask_to_boundary(mask_u8):
    ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    grad = cv2.morphologyEx(mask_u8, cv2.MORPH_GRADIENT, ker)
    return (grad > 0).astype(np.uint8)

def build_gt_mask_and_boundary(jd, label_name, h, w, thickness=2):
    shapes = [sh for sh in jd.get("shapes", []) if sh.get("label") == label_name]
    if not shapes:
        return None, None

    mask = np.zeros((h, w), dtype=np.uint8)
    for sh in shapes:
        st = sh.get("shape_type", "polygon")
        pts = sh.get("points", [])
        if len(pts) < 2:
            continue
        
        pts_xy = [(float(p[0]), float(p[1])) for p in pts]

        if st == "rectangle" and len(pts_xy) >= 2:
            (x1,y1),(x2,y2) = pts_xy[0], pts_xy[1]
            x1,x2 = min(x1,x2), max(x1,x2)
            y1,y2 = min(y1,y2), max(y1,y2)
            pts_xy = [(x1,y1),(x2,y1),(x2,y2),(x1,y2)]

        if st in ("polygon", "rectangle") and len(pts_xy) >= 3:
            pts_xy = ensure_closed(pts_xy)
            if pts_xy is None:
                continue
            poly = pts_to_int(pts_xy, h, w)
            cv2.fillPoly(mask, [poly], 255)

    if int((mask > 0).sum()) > 0:
        return mask, mask_to_boundary(mask)

    bnd = np.zeros((h, w), dtype=np.uint8)
    for sh in shapes:
        pts = sh.get("points", [])
        if len(pts) < 2:
            continue
        
        pts_xy = [(float(p[0]), float(p[1])) for p in pts]
        if len(pts_xy) >= 3:
            pts_xy = ensure_closed(pts_xy) or pts_xy
        
        poly = pts_to_int(pts_xy, h, w)
        cv2.polylines(bnd, [poly], isClosed=False, color=255, thickness=thickness)

    ff = bnd.copy()
    seed = (0, 0)  # Seed point for flood fill
    mask_ff = np.zeros((h + 2, w + 2), np.uint8)
    cv2.floodFill(ff, mask_ff, seedPoint=seed, newVal=128)

    inside = (ff == 0)
    obj = (inside | (ff == 255)).astype(np.uint8) * 255
    
    if int((obj > 0).sum()) == 0:
        return None, None
    
    return obj, (bnd > 0).astype(np.uint8)

def preprocess_exp(gray_u8):
    g = M * np.exp(K * gray_u8.astype(np.float32)) + C
    return np.clip(g, 0, 255).astype(np.uint8)

def canny_edges(gray_u8):
    return cv2.Canny(gray_u8, CANNY_LOW, CANNY_HIGH)

def floodfill_region(gray_u8, seed_xy, thr):
    h, w = gray_u8.shape[:2]
    mask = np.zeros((h + 2, w + 2), np.uint8)  # +2 for flood fill border
    img_ff = gray_u8.copy()
    
    flags = 8 | cv2.FLOODFILL_FIXED_RANGE
    cv2.FLOODFILL_FIXED_RANGE
    cv2.floodFill(img_ff, mask, seedPoint=seed_xy, newVal=255,
                  loDiff=int(thr), upDiff=int(thr), flags=flags)
    
    # Remove border and return binary mask
    return (mask[1:-1, 1:-1] != 0)

def pick_seed_from_gt(gray_u8, gt_mask_u8):
    gt01 = (gt_mask_u8 > 0)
    # Mask out non-GT regions
    vals = gray_u8.copy()
    vals[~gt01] = 0
    # Find coordinates of maximum intensity
    y, x = np.unravel_index(int(np.argmax(vals)), gray_u8.shape)
    return (int(x), int(y))

def coverage(seg_bool, gt_mask_u8):
    gt = (gt_mask_u8 > 0)
    denom = int(gt.sum())
    if denom == 0:
        return 0.0
    inter = int((seg_bool & gt).sum())
    return float(inter / denom)

def bg_false_edge_density(edge_u8, gt_mask_u8, exclude_dilate=10):
    gt = (gt_mask_u8 > 0).astype(np.uint8) * 255
    k = int(exclude_dilate)
    if k % 2 == 0:
        k += 1  # Ensure odd kernel size
    ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k, k))
    gt_d = cv2.dilate(gt, ker, iterations=1)
    
    bg = (gt_d == 0)
    denom = int(bg.sum())
    if denom == 0:
        return 0.0
    
    e = (edge_u8 > 0)
    return float((e & bg).sum() / denom)

def disk_kernel(radius):
    r = int(radius)
    k = 2 * r + 1
    return cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k, k))

def boundary_band_mask(gt_boundary01_u8, band=8):
    gt_b = gt_boundary01_u8 * 255
    ker = disk_kernel(band)
    # Dilate boundary to create band
    dilated = cv2.dilate(gt_b, ker, iterations=1)
    return (dilated > 0).astype(np.uint8)

def boundary_f1_dilate(pred_edge_u8, gt_boundary01_u8, tol=2, pred_mask01=None):
    pred = (pred_edge_u8 > 0).astype(np.uint8)
    gt_b = gt_boundary01_u8.astype(np.uint8)

    if pred_mask01 is not None:
        pred = pred & pred_mask01.astype(np.uint8)

    n_pred = int(pred.sum())
    n_gt = int(gt_b.sum())
    
    if n_pred == 0 or n_gt == 0:
        return 0.0

    ker = disk_kernel(tol)
    gt_d = cv2.dilate(gt_b * 255, ker, iterations=1) > 0
    pred_d = cv2.dilate(pred * 255, ker, iterations=1) > 0

    # Calculate true positives
    hit_pred = int((pred.astype(bool) & gt_d).sum())
    hit_gt = int((gt_b.astype(bool) & pred_d).sum())

    prec = hit_pred / n_pred if n_pred > 0 else 0.0
    rec = hit_gt / n_gt if n_gt > 0 else 0.0

    if (prec + rec) < 1e-12:
        return 0.0
    return 2 * prec * rec / (prec + rec)

def med_p90(vals):
    vals = np.asarray(vals, dtype=np.float64)
    # Filter out non-finite values
    vals = vals[np.isfinite(vals)]
    if vals.size == 0:
        return (np.nan, np.nan)
    return (float(np.median(vals)), float(np.percentile(vals, 90)))


def main():
    json_files = sorted(glob.glob(os.path.join(LABEL_DIR, "*.json")))
    n_total = len(json_files)

    if n_total == 0:
        raise FileNotFoundError(f"No JSON files found in directory: {LABEL_DIR}")

    rows = []  
    cov_b_list = []  
    cov_p_list = []  
    f1_band_b_list = []  
    f1_band_p_list = []
    f1_full_b_list = []  
    f1_full_p_list = []  
    bg_b_list = []  
    bg_p_list = [] 

    t_start = time.time()
    last_pct = -1

    for idx, json_path in enumerate(json_files, start=1):
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                jd = json.load(f)
        except Exception as e:
            print(f"[WARNING] Failed to load JSON: {json_path} - {str(e)}")
            continue

        img, img_path = load_img(jd, json_path)
        if img is None:
            print(f"[WARNING] Failed to load image for: {json_path}")
            continue

        gray = to_gray_u8(img)
        h, w = gray.shape[:2]

        label_used = resolve_label_name(jd, LABEL_NAME)
        if label_used is None:
            print(f"[WARNING] No valid labels found in: {json_path}")
            continue

        gt_mask, gt_boundary = build_gt_mask_and_boundary(jd, label_used, h, w, BOUNDARY_THICKNESS)
        if gt_mask is None or int((gt_mask > 0).sum()) == 0:
            print(f"[WARNING] Invalid GT mask for: {json_path}")
            continue

        seed = pick_seed_from_gt(gray, gt_mask)
        band_mask = boundary_band_mask(gt_boundary, EVAL_BAND)

        edges_baseline = canny_edges(gray)
        seg_baseline = floodfill_region(gray, seed, GROW_THRESH)
        cov_baseline = coverage(seg_baseline, gt_mask)
        bg_baseline = bg_false_edge_density(edges_baseline, gt_mask, BG_EXCLUDE_DILATE)
        f1_full_baseline = boundary_f1_dilate(edges_baseline, gt_boundary, BOUNDARY_TOL, None)
        f1_band_baseline = boundary_f1_dilate(edges_baseline, gt_boundary, BOUNDARY_TOL, band_mask)

        median_img = cv2.medianBlur(gray, MEDIAN_KSIZE)
        preprocessed_img = preprocess_exp(median_img)
        edges_pre = canny_edges(preprocessed_img)
        seg_pre = floodfill_region(preprocessed_img, seed, GROW_THRESH)
        cov_pre = coverage(seg_pre, gt_mask)
        bg_pre = bg_false_edge_density(edges_pre, gt_mask, BG_EXCLUDE_DILATE)
        f1_full_pre = boundary_f1_dilate(edges_pre, gt_boundary, BOUNDARY_TOL, None)
        f1_band_pre = boundary_f1_dilate(edges_pre, gt_boundary, BOUNDARY_TOL, band_mask)

        img_filename = os.path.basename(img_path) if img_path else os.path.splitext(os.path.basename(json_path))[0]
        rows.append([
            img_filename,
            label_used,
            seed[0],
            seed[1],
            cov_baseline,
            cov_pre,
            f1_band_baseline,
            f1_band_pre,
            f1_full_baseline,
            f1_full_pre,
            bg_baseline,
            bg_pre
        ])

        cov_b_list.append(cov_baseline)
        cov_p_list.append(cov_pre)
        f1_band_b_list.append(f1_band_baseline)
        f1_band_p_list.append(f1_band_pre)
        f1_full_b_list.append(f1_full_baseline)
        f1_full_p_list.append(f1_full_pre)
        bg_b_list.append(bg_baseline)
        bg_p_list.append(bg_pre)

        current_pct = int(idx * 100 / n_total)
        if current_pct >= last_pct + PROGRESS_STEP or idx == n_total:
            elapsed = time.time() - t_start
            print(f"[PROGRESS] {current_pct}% ({idx}/{n_total}) | Elapsed: {elapsed:.1f}s")
            last_pct = current_pct

    try:
        with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
            csv_writer = csv.writer(f)
            # Write header
            csv_writer.writerow([
                "image_filename",
                "label_used",
                "seed_x",
                "seed_y",
                "coverage_baseline",
                "coverage_preprocessed",
                "f1_band_baseline",
                "f1_band_preprocessed",
                "f1_full_baseline",
                "f1_full_preprocessed",
                "bg_false_edge_density_baseline",
                "bg_false_edge_density_preprocessed"
            ])
            csv_writer.writerows(rows)
        print(f"[SUCCESS] CSV file saved to: {OUT_CSV}")
    except Exception as e:
        raise RuntimeError(f"Failed to save CSV file: {str(e)}")

    try:
        with open(OUT_TXT, "w", encoding="utf-8") as f:
            # Write basic information
            f.write("=== Evaluation Summary ===\n")
            f.write(f"Total JSON files: {n_total}\n")
            f.write(f"Successfully evaluated: {len(rows)}\n")
            f.write(f"Elapsed time: {time.time() - t_start:.1f} seconds\n\n")
            
            # Write parameters
            f.write("=== Algorithm Parameters ===\n")
            f.write(f"Canny edge detection: low={CANNY_LOW}, high={CANNY_HIGH}\n")
            f.write(f"Boundary tolerance: {BOUNDARY_TOL} pixels\n")
            f.write(f"Evaluation band width: {EVAL_BAND} pixels\n")
            f.write(f"Background exclude dilation: {BG_EXCLUDE_DILATE} pixels\n")
            f.write(f"Median filter kernel size: {MEDIAN_KSIZE}\n")
            f.write(f"MKC transformation: M={M}, K={K}, C={C}\n")
            f.write(f"Region growing threshold: {GROW_THRESH}\n")
            f.write(f"Boundary thickness for polyline: {BOUNDARY_THICKNESS}\n\n")
            
            f.write("=== Metric Summary (Median / 90th Percentile) ===\n")
            
            f.write("\n1. Region Growing Coverage (higher is better)\n")
            f.write(f"   Baseline: {med_p90(cov_b_list)}\n")
            f.write(f"   Preprocessed: {med_p90(cov_p_list)}\n")
            
            f.write("\n2. Boundary F1 Score (Band Region, higher is better)\n")
            f.write(f"   Baseline: {med_p90(f1_band_b_list)}\n")
            f.write(f"   Preprocessed: {med_p90(f1_band_p_list)}\n")
            
            f.write("\n3. Boundary F1 Score (Full Image, higher is better)\n")
            f.write(f"   Baseline: {med_p90(f1_full_b_list)}\n")
            f.write(f"   Preprocessed: {med_p90(f1_full_p_list)}\n")
            
            f.write("\n4. Background False Edge Density (lower is better)\n")
            f.write(f"   Baseline: {med_p90(bg_b_list)}\n")
            f.write(f"   Preprocessed: {med_p90(bg_p_list)}\n")
        
        print(f"[SUCCESS] Summary TXT file saved to: {OUT_TXT}")
    except Exception as e:
        raise RuntimeError(f"Failed to save summary TXT file: {str(e)}")

    print("\n[FINISHED] Evaluation completed successfully!")
    print(f"- Processed {len(rows)} out of {n_total} files")
    print(f"- CSV output: {OUT_CSV}")
    print(f"- Summary output: {OUT_TXT}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[ERROR] Evaluation failed: {str(e)}")
        exit(1)